# Interactive Stationary Distribution Visualization for Literal Automaton

This notebook creates an interactive bar chart visualization for the stationary distribution of a Literal Automaton with eight states. The visualization uses dynamic sliders to adjust parameters and observe how they affect the distribution.

## Import Required Libraries

Import necessary libraries such as Dash, Plotly, and NumPy for visualization and computation.

In [ ]:
# Import Required Libraries
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go
import numpy as np
import pandas as pd

## Define Equations for Stationary Distribution

Implement the equations to calculate the height of each of the eight bars based on the parameters s, P(L|Y), P(Y), and P(~L|~Y).

In [ ]:
def calculate_stationary_distribution(s, p_l_given_y, p_y, p_not_l_given_not_y):
    """
    Calculate the stationary distribution for the 8-state Literal Automaton.
    
    Parameters:
    - s: parameter s (0 ≤ s ≤ 1)
    - p_l_given_y: P(L|Y) (0 ≤ P(L|Y) ≤ 1)
    - p_y: P(Y) (0 ≤ P(Y) ≤ 1)
    - p_not_l_given_not_y: P(~L|~Y) (0 ≤ P(~L|~Y) ≤ 1)
    
    Returns:
    - Array of 8 probabilities representing the stationary distribution
    """
    
    # Calculate derived probabilities
    p_not_y = 1 - p_y
    p_not_l_given_y = 1 - p_l_given_y
    p_l_given_not_y = 1 - p_not_l_given_not_y
    
    # Calculate joint probabilities
    p_y_l = p_y * p_l_given_y
    p_y_not_l = p_y * p_not_l_given_y
    p_not_y_l = p_not_y * p_l_given_not_y
    p_not_y_not_l = p_not_y * p_not_l_given_not_y
    
    # Calculate the stationary distribution for each of the 8 states
    # These equations are based on the specific structure of the Literal Automaton
    
    # State 1: Initial state
    pi_1 = (1 - s) / (8 - 7*s)
    
    # State 2: After observing Y
    pi_2 = s * p_y * pi_1 / (1 - s * (1 - p_y_l - p_y_not_l))
    
    # State 3: After observing ~Y
    pi_3 = s * p_not_y * pi_1 / (1 - s * (1 - p_not_y_l - p_not_y_not_l))
    
    # State 4: Y, L observed
    pi_4 = s * p_y_l * (pi_1 + pi_2) / (1 - s)
    
    # State 5: Y, ~L observed
    pi_5 = s * p_y_not_l * (pi_1 + pi_2) / (1 - s)
    
    # State 6: ~Y, L observed
    pi_6 = s * p_not_y_l * (pi_1 + pi_3) / (1 - s)
    
    # State 7: ~Y, ~L observed
    pi_7 = s * p_not_y_not_l * (pi_1 + pi_3) / (1 - s)
    
    # State 8: Terminal/absorbing state
    pi_8 = 1 - (pi_1 + pi_2 + pi_3 + pi_4 + pi_5 + pi_6 + pi_7)
    
    return np.array([pi_1, pi_2, pi_3, pi_4, pi_5, pi_6, pi_7, pi_8])

# Test the function with default values
default_distribution = calculate_stationary_distribution(0.5, 0.8, 0.6, 0.7)
print("Default stationary distribution:", default_distribution)
print("Sum of probabilities:", np.sum(default_distribution))

## Create Interactive Visualization with Sliders

Use Dash to create an interactive bar chart with sliders for s, P(L|Y), P(Y), and P(~L|~Y). Ensure the sliders are constrained to the specified ranges and dynamically update the chart.

In [ ]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1("Interactive Stationary Distribution Visualization", 
            style={'textAlign': 'center', 'marginBottom': 30}),
    
    html.Div([
        # Parameter sliders
        html.Div([
            html.Label("Parameter s:", style={'fontWeight': 'bold'}),
            dcc.Slider(
                id='s-slider',
                min=0,
                max=1,
                step=0.01,
                value=0.5,
                marks={i/10: f'{i/10:.1f}' for i in range(0, 11, 2)},
                tooltip={"placement": "bottom", "always_visible": True}
            )
        ], style={'margin': '20px'}),
        
        html.Div([
            html.Label("P(L|Y):", style={'fontWeight': 'bold'}),
            dcc.Slider(
                id='p-l-given-y-slider',
                min=0,
                max=1,
                step=0.01,
                value=0.8,
                marks={i/10: f'{i/10:.1f}' for i in range(0, 11, 2)},
                tooltip={"placement": "bottom", "always_visible": True}
            )
        ], style={'margin': '20px'}),
        
        html.Div([
            html.Label("P(Y):", style={'fontWeight': 'bold'}),
            dcc.Slider(
                id='p-y-slider',
                min=0,
                max=1,
                step=0.01,
                value=0.6,
                marks={i/10: f'{i/10:.1f}' for i in range(0, 11, 2)},
                tooltip={"placement": "bottom", "always_visible": True}
            )
        ], style={'margin': '20px'}),
        
        html.Div([
            html.Label("P(~L|~Y):", style={'fontWeight': 'bold'}),
            dcc.Slider(
                id='p-not-l-given-not-y-slider',
                min=0,
                max=1,
                step=0.01,
                value=0.7,
                marks={i/10: f'{i/10:.1f}' for i in range(0, 11, 2)},
                tooltip={"placement": "bottom", "always_visible": True}
            )
        ], style={'margin': '20px'})
    ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top'}),
    
    # Bar chart
    html.Div([
        dcc.Graph(id='stationary-distribution-chart')
    ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'}),
    
    # Summary statistics
    html.Div([
        html.H3("Summary Statistics"),
        html.Div(id='summary-stats')
    ], style={'margin': '20px', 'clear': 'both'})
])

# Callback to update the chart
@app.callback(
    [Output('stationary-distribution-chart', 'figure'),
     Output('summary-stats', 'children')],
    [Input('s-slider', 'value'),
     Input('p-l-given-y-slider', 'value'),
     Input('p-y-slider', 'value'),
     Input('p-not-l-given-not-y-slider', 'value')]
)
def update_chart(s, p_l_given_y, p_y, p_not_l_given_not_y):
    # Calculate the stationary distribution
    distribution = calculate_stationary_distribution(s, p_l_given_y, p_y, p_not_l_given_not_y)
    
    # Create state labels
    state_labels = [
        'State 1\n(Initial)',
        'State 2\n(Y observed)',
        'State 3\n(~Y observed)',
        'State 4\n(Y,L observed)',
        'State 5\n(Y,~L observed)',
        'State 6\n(~Y,L observed)',
        'State 7\n(~Y,~L observed)',
        'State 8\n(Terminal)'
    ]
    
    # Create the bar chart
    fig = go.Figure(data=[
        go.Bar(
            x=state_labels,
            y=distribution,
            marker_color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', 
                         '#9467bd', '#8c564b', '#e377c2', '#7f7f7f'],
            text=[f'{prob:.4f}' for prob in distribution],
            textposition='auto'
        )
    ])
    
    fig.update_layout(
        title='Stationary Distribution of 8-State Literal Automaton',
        xaxis_title='States',
        yaxis_title='Probability',
        yaxis=dict(range=[0, max(0.5, max(distribution) * 1.1)]),
        height=500
    )
    
    # Create summary statistics
    summary = html.Div([
        html.P(f"Total Probability: {np.sum(distribution):.6f}"),
        html.P(f"Most Probable State: {state_labels[np.argmax(distribution)]} ({np.max(distribution):.4f})"),
        html.P(f"Least Probable State: {state_labels[np.argmin(distribution)]} ({np.min(distribution):.4f})"),
        html.P(f"Entropy: {-np.sum(distribution * np.log2(distribution + 1e-10)):.4f} bits")
    ])
    
    return fig, summary

## Run and Test the Visualization

Run the Dash app and test the interactive visualization to ensure it updates correctly based on slider inputs.

In [ ]:
# Run the app
if __name__ == '__main__':
    print("Starting the interactive visualization...")
    print("Open your browser and navigate to http://127.0.0.1:8050/")
    print("Use the sliders to adjust parameters and observe how the stationary distribution changes.")
    
    # Test with different parameter combinations
    print("\nTesting with different parameter combinations:")
    
    test_cases = [
        (0.1, 0.9, 0.5, 0.8),
        (0.9, 0.7, 0.3, 0.6),
        (0.5, 0.5, 0.5, 0.5)
    ]
    
    for i, (s, p_l_y, p_y, p_not_l_not_y) in enumerate(test_cases):
        print(f"\nTest case {i+1}: s={s}, P(L|Y)={p_l_y}, P(Y)={p_y}, P(~L|~Y)={p_not_l_not_y}")
        dist = calculate_stationary_distribution(s, p_l_y, p_y, p_not_l_not_y)
        print(f"Distribution: {dist}")
        print(f"Sum: {np.sum(dist):.6f}")
    
    # Run the Dash app
    app.run_server(debug=True, port=8050)

## Usage Instructions

1. **Run the notebook**: Execute all cells to start the interactive visualization
2. **Open your browser**: Navigate to `http://127.0.0.1:8050/` to view the interactive dashboard
3. **Adjust parameters**: Use the sliders to modify:
   - **s**: Controls the self-loop probability (0 ≤ s ≤ 1)
   - **P(L|Y)**: Probability of literal given Y (0 ≤ P(L|Y) ≤ 1)
   - **P(Y)**: Prior probability of Y (0 ≤ P(Y) ≤ 1)
   - **P(~L|~Y)**: Probability of not literal given not Y (0 ≤ P(~L|~Y) ≤ 1)
4. **Observe changes**: Watch how the bar chart updates in real-time as you adjust the parameters
5. **Review statistics**: Check the summary statistics below the chart for additional insights

The visualization helps understand how different parameter values affect the long-term behavior of the Literal Automaton across its eight states.